In [11]:
data = pd.read_csv('timeseries.csv')

# Configurando bins (faixas de intensidade)
bins = [0, 25, 50, 75, 100]

# Histograma para aceleração
pp.hist(data['acceleration'], bins=bins, alpha=0.5, label='Aceleração', color='green')

# Histograma para frenagem
pp.hist(data['brake'], bins=bins, alpha=0.5, label='Frenagem', color='red')

# Configuração do gráfico
pp.xlabel('Intensidade (%)')
pp.ylabel('Frequência')
pp.title('Histograma de Frenagem e Aceleração')
pp.legend(loc='upper right')

# Exibir o gráfico
pp.show()

KeyError: 'acceleration'

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as pp

In [ ]:
df = pd.read_csv('telemetry-rio-5-laps.csv').loc[:'Index', ["current_engine_rpm", "wheel_rotation_speed_front_left", "wheel_rotation_speed_front_right", "wheel_rotation_speed_rear_left", "wheel_rotation_speed_rear_right", "wheel_on_rumble_strip_front_left", "wheel_on_rumble_strip_front_right", "wheel_on_rumble_strip_rear_left", "wheel_on_rumble_strip_rear_right", "wheel_in_puddle_depth_front_left", "wheel_in_puddle_depth_front_right", "wheel_in_puddle_depth_rear_left", "wheel_in_puddle_depth_rear_right", "tire_slip_rotation_front_left", "tire_slip_rotation_front_right", "tire_slip_rotation_rear_left", "tire_slip_rotation_rear_right", "tire_slip_angle_front_left", "tire_slip_angle_front_right", "tire_slip_angle_rear_left", "tire_slip_angle_rear_right", "tire_combined_slip_front_left", "tire_combined_slip_front_right", "tire_combined_slip_rear_left", "tire_combined_slip_rear_right", "tire_temp_front_left", "tire_temp_front_right", "tire_temp_rear_left", "tire_temp_rear_right", "normalized_suspension_travel_front_left", "normalized_suspension_travel_front_right", "normalized_suspension_travel_rear_left", "normalized_suspension_travel_rear_right", "suspension_travel_meters_front_left", "suspension_travel_meters_front_right", "suspension_travel_meters_rear_left", "suspension_travel_meters_rear_right", "position_x", "position_y", "position_z", "acceleration_x", "acceleration_y", "acceleration_z", "velocity_x", "velocity_y", "velocity_z", "angular_velocity_x","angular_velocity_y", "angular_velocity_z", "yaw", "pitch", "roll", "speed", "power", "torque", "boost", "fuel", "distance_traveled", "acceleration", "brake", "clutch", "handbrake", "gear", "steer", "lap_number", "best_lap_time", "last_lap_time", "current_lap_time", "current_race_time", "race_position"]]
df['lap_number'] = df['lap_number'].replace(4, 5)
df['lap_number'] = df['lap_number'].replace(3, 4)
df['lap_number'] = df['lap_number'].replace(2, 3)
df['lap_number'] = df['lap_number'].replace(1, 2)
df['lap_number'] = df['lap_number'].replace(0, 1)
df['speed'] = df['speed'] * 3.6
df.to_csv('timeseries.csv', decimal=",", sep=";")